In [320]:
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModel
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Subset
import datasets

In [321]:
from dataclasses import dataclass

@dataclass
class Task:
    id : int
    num_labels: int
    type: str

In [322]:
data = pd.read_excel('/home/elson/topk3_minilm.xlsx',engine='openpyxl')
df= data.dropna(subset=['label','CATEGORY'])
label_encoder = LabelEncoder()
label_encoder2 = LabelEncoder()
claims = df.claim.tolist()
labels = df.label.tolist()
labels = label_encoder.fit_transform(labels)
evidence_1 = df.top_1_minilm_ce.to_list()
evidence_2 = df.top_2_minilm_ce.to_list()
evidence_3 = df.top_3_minilm_ce.to_list()
category = df.CATEGORY.tolist()
category = label_encoder2.fit_transform(category)

In [323]:
print(label_encoder2.classes_)

['Blood' 'Bone health' 'COVID' 'COVID-19' 'Cancer' 'Cardiovascular Health'
 'Dental Health' 'Diabetes' 'Ear' 'Eye' 'Fitness'
 'Gastrointestinal Health' 'General Health' 'Hair' 'Kidney' "Men's health"
 'Mental Health' 'Muscles' 'Neurological health' 'Skin' 'Throat'
 'Vascular' "Women' s Health"]


In [324]:
from sklearn.model_selection import train_test_split

# Perform the split
train_premises, test_premises, train_hypothesis1, test_hypothesis1,train_hypothesis2, test_hypothesis2, train_hypothesis3, test_hypothesis3, train_labels, test_labels = train_test_split(
    claims, evidence_1,evidence_2,evidence_3, labels, test_size=0.2, random_state=42)

In [325]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

class MediClaimVeracityDataset(Dataset):
    def __init__(self, premises, hypothesis1, hypothesis2, hypothesis3, labels, tokenizer_name='allenai/scibert_scivocab_uncased'):
        self.premises = premises
        self.hypothesis1 = hypothesis1
        self.hypothesis2 = hypothesis2
        self.hypothesis3 = hypothesis3
        self.labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Corrected: Use `.join` properly to concatenate the hypotheses with the premises.
        separator = self.tokenizer.sep_token
        grouped = self.premises[idx] + separator + separator.join([self.hypothesis1[idx], self.hypothesis2[idx], self.hypothesis3[idx]])
        
        tokenized_input = self.tokenizer(
            text=grouped,
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt')
        tokenized_input = {key: val.squeeze(0) for key, val in tokenized_input.items()}
        
        # Adding labels and task_id to the returned dictionary.
        tokenized_input['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        tokenized_input['task_ids'] = torch.tensor(0)  # Adding task_id as 0 for all instances
        
        return tokenized_input

def load_entailment_dataset(id=0):
    trainV_dataset = MediClaimVeracityDataset(train_premises, train_hypothesis1, train_hypothesis2, train_hypothesis3, train_labels)
    valV_dataset = MediClaimVeracityDataset(test_premises, test_hypothesis1, test_hypothesis2, test_hypothesis3, test_labels)
    logging_steps = len(test_premises) // 16

        # Wrap the subsets with DataLoader
    #trainV_loader = DataLoader(trainV_dataset, batch_size=16, shuffle=True)
    #valV_loader = DataLoader(valV_dataset, batch_size=16, shuffle=False)
    num_labels = len(train_premises)
    task_info = Task(
            id, num_labels=num_labels, type="ver_classification"
        )

    return trainV_dataset, valV_dataset, task_info

In [326]:
load_entailment_dataset()

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:17,724 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:17,902 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:17,905 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

(<__main__.MediClaimVeracityDataset at 0x7fbae33f4b70>,
 Task(id=0, num_labels=619, type='ver_classification'))

In [327]:
train_stat, test_stat, train_categories, test_categories = train_test_split(
    claims, category, test_size=0.2, random_state=42)

from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

class MediClaimCategoryDataset(Dataset):
    def __init__(self, claims, category, tokenizer_name='allenai/scibert_scivocab_uncased'):
        self.claims = claims
        self.category = category
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    def __len__(self):
        return len(self.category)
    
    def __getitem__(self, idx):
        tokenized_input = self.tokenizer(
            text=self.claims[idx],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt')
        tokenized_input = {key: val.squeeze(0) for key, val in tokenized_input.items()}
        
        # Adding 'labels' to align with the format in the `forward` method and MultiTaskModel expectations
        tokenized_input['labels'] = torch.tensor(self.category[idx], dtype=torch.long)
        # Adding task_id as 1 for all instances to identify this as category classification
        tokenized_input['task_ids'] = torch.tensor(1)
        
        return tokenized_input

def load_category_dataset(id=1):
    trainC_dataset = MediClaimCategoryDataset(train_stat,train_categories)
    valC_dataset = MediClaimCategoryDataset(test_stat,test_categories)
    logging_steps = len(train_stat) // 16

        # Wrap the subsets with DataLoader
#     trainC_loader = DataLoader(trainC_dataset, batch_size=16, shuffle=True)
#     valC_loader = DataLoader(valC_dataset, batch_size=16, shuffle=False)
    num_labels = len(train_stat)
    task_info = Task(
            id, num_labels=num_labels, type="cat_classification"
        )

    return trainC_dataset, valC_dataset, task_info

In [328]:
load_category_dataset()

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:24,732 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:25,115 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:25,119 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

(<__main__.MediClaimCategoryDataset at 0x7fb81c2a71d0>,
 Task(id=1, num_labels=619, type='cat_classification'))

In [329]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CombinedDataset(Dataset):
    def __init__(self, dataset1, dataset2):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        self.total_size = len(dataset1) + len(dataset2)
        
        # Create an array that indicates which dataset each index should fetch from
        self.dataset_labels = np.array([0]*len(dataset1) + [1]*len(dataset2))
        # Shuffle to mix dataset indices
        np.random.shuffle(self.dataset_labels)

    def __len__(self):
        return self.total_size
    
    def __getitem__(self, idx):
        if self.dataset_labels[idx] == 0:
            # Fetch from dataset1
            return self.dataset1[idx % len(self.dataset1)]
        else:
            # Fetch from dataset2
            return self.dataset2[idx % len(self.dataset2)]

In [330]:
def load_datasets():
    (
        veracity_classification_train_dataset,
        veracity_classification_validation_dataset,
        veracity_classification_task,
    ) = load_entailment_dataset(0)
    (
        cat_classification_train_dataset,
        cat_classification_validation_dataset,
        cat_classification_task,
    ) = load_category_dataset(1)

    # Merge train datasets
    train_combined_dataset = CombinedDataset(veracity_classification_train_dataset, cat_classification_train_dataset)
    val_combined_dataset = CombinedDataset(veracity_classification_validation_dataset, cat_classification_validation_dataset)

    
    tasks = [veracity_classification_task, cat_classification_task]
    return tasks, train_combined_dataset,val_combined_dataset

In [331]:
load_datasets()

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:31,263 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:31,647 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:31,650 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:38,064 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:38,236 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:38,239 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

([Task(id=0, num_labels=619, type='ver_classification'),
  Task(id=1, num_labels=619, type='cat_classification')],
 <__main__.CombinedDataset at 0x7fb81c2a7710>)

In [332]:
import torch
from torch import nn
from transformers import AutoModel
from typing import List, Dict

class MultiTaskModel(nn.Module):
    def __init__(self, encoder_name_or_path, tasks: List):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name_or_path)
        self.output_heads = nn.ModuleDict()

        for task in tasks:
            decoder = self._create_output_head(self.encoder.config.hidden_size, task)
            self.output_heads[str(task.id)] = decoder

    @staticmethod
    def _create_output_head(encoder_hidden_size: int, task):
        # Placeholder for actual implementation
        if task.type == "ver_classification":
            return VeracityClassificationHead(encoder_hidden_size, task.num_labels)
        elif task.type == "cat_classification":
            return CategoryClassificationHead(encoder_hidden_size, task.num_labels)
        else:
            raise NotImplementedError()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        task_ids=None,
        **kwargs,
    ):

        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        sequence_output, pooled_output = outputs[:2]

        unique_task_ids_list = torch.unique(task_ids).tolist()

        loss_list = []
        logits = None
        for unique_task_id in unique_task_ids_list:

            task_id_filter = task_ids == unique_task_id
            logits, task_loss = self.output_heads[str(unique_task_id)].forward(
                sequence_output[task_id_filter],
                pooled_output[task_id_filter],
                labels=None if labels is None else labels[task_id_filter],
                attention_mask=attention_mask[task_id_filter],
            )

            if labels is not None:
                loss_list.append(task_loss)

        # logits are only used for eval. and in case of eval the batch is not multi task
        # For training only the loss is used
        outputs = (logits, outputs[2:])

        if loss_list:
            loss = torch.stack(loss_list)
            outputs = (loss.mean(),) + outputs

        return outputs

In [333]:
class VeracityClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels, dropout_p=0.1):
        super().__init__()
        self.num_labels = num_labels
        self.dropout = nn.Dropout(dropout_p)
        self.classifier = nn.Linear(hidden_size, num_labels)

        self._init_weights()

    def forward(self, sequence_output, pooled_output, labels=None, **kwargs):
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if labels.dim() != 1:
                # Remove padding
                labels = labels[:, 0]

            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                logits.view(-1, self.num_labels), labels.long().view(-1)
            )

        return logits, loss

    def _init_weights(self):
        self.classifier.weight.data.normal_(mean=0.0, std=0.02)
        if self.classifier.bias is not None:
            self.classifier.bias.data.zero_()

In [334]:
class CategoryClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels, dropout_p=0.1):
        super().__init__()
        self.num_labels = num_labels
        self.dropout = nn.Dropout(dropout_p)
        self.classifier = nn.Linear(hidden_size, num_labels)

        self._init_weights()

    def forward(self, sequence_output, pooled_output, labels=None, **kwargs):
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if labels.dim() != 1:
                # Remove padding
                labels = labels[:, 0]

            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                logits.view(-1, self.num_labels), labels.long().view(-1)
            )

        return logits, loss

    def _init_weights(self):
        self.classifier.weight.data.normal_(mean=0.0, std=0.02)
        if self.classifier.bias is not None:
            self.classifier.bias.data.zero_()

In [335]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(pred: EvalPrediction):
    # Extract predictions and true labels
    preds = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = np.argmax(preds, axis=1) if preds.ndim > 1 else preds.squeeze()
    
    true_labels = pred.label_ids
    
    # Ensure predictions and labels are aligned
    # This is a simplistic fix; for real scenarios, you might need more complex handling
    min_len = min(len(preds), len(true_labels))
    preds, true_labels = preds[:min_len], true_labels[:min_len]
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted', labels=np.unique(preds))
    precision = precision_score(true_labels, preds, average='weighted', labels=np.unique(preds))
    recall = recall_score(true_labels, preds, average='weighted', labels=np.unique(preds))

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}


In [336]:
import logging
import sys
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",datefmt="%m/%d/%Y %H:%M:%S",handlers=[logging.StreamHandler(sys.stdout)],)
import transformers
log_level = training_args.get_process_log_level()
logger = logging.getLogger(__name__)
logger.setLevel(log_level)
transformers.logging.set_verbosity_info()
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

   

In [337]:
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:44,498 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:44,852 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:44,855 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

In [338]:
tasks, train_dataset,val_dataset=load_datasets()

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:48,070 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:48,442 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:48,445 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

[INFO|tokenization_auto.py:344] 2024-03-25 17:31:55,702 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2024-03-25 17:31:56,081 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:31:56,084 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pa

In [339]:
model = MultiTaskModel(model_name, tasks)

[INFO|configuration_utils.py:654] 2024-03-25 17:32:01,569 >> loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/elson/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
[INFO|configuration_utils.py:690] 2024-03-25 17:32:01,572 >> Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size"

In [340]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer)

[INFO|trainer.py:453] 2024-03-25 17:32:03,616 >> Using amp half precision backend


In [341]:
logging_steps = len(train_dataset)//16
training_args = TrainingArguments(
    output_dir="/home/elson/multi-task/scibert",  # Specify your desired output directory
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    dataloader_pin_memory=True,
    dataloader_num_workers=4,
    gradient_accumulation_steps=2,
    fp16=True,
    num_train_epochs=40,
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_ratio=0.06,
    evaluation_strategy="epoch",
    save_strategy="no",
    disable_tqdm=False,
    logging_steps=logging_steps,  # Make sure logging_steps is defined, e.g., 50
    push_to_hub=False,
    )

trainer = Trainer(
    model=model,  # Make sure your model is specified here
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    )
    # Start the training process
trainer.train()

[INFO|training_args.py:1023] 2024-03-25 17:32:03,666 >> PyTorch: setting up devices
[INFO|training_args.py:886] 2024-03-25 17:32:03,669 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|trainer.py:453] 2024-03-25 17:32:03,676 >> Using amp half precision backend
/home/elson/factcheck/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
[INFO|trainer.py:1290] 2024-03-25 17:32:03,687 >> ***** Running training *****
[INFO|trainer.py:1291] 2024-03-25 17:32:03,688 >>   Num examples = 1238
[INFO|trainer.py:1292] 2024-03-25 17:32:0

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,6.546908,0.000000,0.000000,0.000000,0.000000
2,No log,6.114854,0.000000,0.000000,0.000000,0.000000
3,No log,5.755576,0.000000,0.000000,0.000000,0.000000
4,No log,5.321622,0.006452,0.076923,0.052632,0.142857
5,No log,4.947538,0.038710,0.108108,0.057692,0.857143
6,No log,4.656958,0.051613,0.084634,0.117060,0.320000
7,No log,4.409511,0.064516,0.123598,0.074351,0.400000
8,5.499100,4.198694,0.103226,0.163310,0.094545,0.640000
9,5.499100,4.013943,0.116129,0.151579,0.084706,0.720000
10,5.499100,3.851131,0.116129,0.208092,0.116129,1.000000


[INFO|trainer.py:2416] 2024-03-25 17:32:10,049 >> ***** Running Evaluation *****
[INFO|trainer.py:2418] 2024-03-25 17:32:10,050 >>   Num examples = 310
[INFO|trainer.py:2421] 2024-03-25 17:32:10,051 >>   Batch size = 16
/home/elson/factcheck/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elson/factcheck/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:2416] 2024-03-25 17:32:17,691 >> ***** Running Evaluation *****
[INFO|trainer.py:2418] 2024-03-25 17:32:17,692 >>   Num examples = 310
[INFO|trainer.py:2421] 2024-03-25 17:32:17,

[INFO|trainer.py:2416] 2024-03-25 17:33:39,782 >> ***** Running Evaluation *****
[INFO|trainer.py:2418] 2024-03-25 17:33:39,785 >>   Num examples = 310
[INFO|trainer.py:2421] 2024-03-25 17:33:39,786 >>   Batch size = 16
/home/elson/factcheck/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:2416] 2024-03-25 17:33:47,303 >> ***** Running Evaluation *****
[INFO|trainer.py:2418] 2024-03-25 17:33:47,305 >>   Num examples = 310
[INFO|trainer.py:2421] 2024-03-25 17:33:47,307 >>   Batch size = 16
/home/elson/factcheck/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, bu

[INFO|trainer.py:2421] 2024-03-25 17:35:39,641 >>   Batch size = 16
/home/elson/factcheck/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:2416] 2024-03-25 17:35:47,014 >> ***** Running Evaluation *****
[INFO|trainer.py:2418] 2024-03-25 17:35:47,016 >>   Num examples = 310
[INFO|trainer.py:2421] 2024-03-25 17:35:47,017 >>   Batch size = 16
/home/elson/factcheck/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:2416] 2024-03-25 17:35:54,372 >> ***** Running Evaluation *****
[INFO|trainer.py:2418] 2024-03-25 17:35:54,373 >>   Num example

TrainOutput(global_step=400, training_loss=3.4750553798675536, metrics={'train_runtime': 299.6624, 'train_samples_per_second': 165.253, 'train_steps_per_second': 1.335, 'total_flos': 0.0, 'train_loss': 3.4750553798675536, 'epoch': 40.0})